# Imports

In [1]:
# import modules
import pandas as pd
import numpy as np
import re
from ast import literal_eval
import spotipy

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:_.2f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [2]:
# helper functions

def find_all_tracks(track_title, artist_name):
    """ returns list of lists ['id', 'song', 'artist'] """
    track_info = spotify.search(q='artist:' + artist_name + ' track:' + track_title, type='track')
    
    if track_info['tracks']['items'] == []:  # if track doesn't exist on Spotify
        return 'MISSING'
    else:
        all_tracks = []
        number_of_results = len(track_info['tracks']['items'])
        
        # check if there is a better match
        for i in range(number_of_results):
            track_id = track_info['tracks']['items'][i]['id']
            artist_name = track_info['tracks']['items'][i]['artists'][0]['name']
            song_name = track_info['tracks']['items'][i]['name']
            all_tracks.append([track_id, song_name, artist_name])
        
        # if we made it through the loop without returning, note 'MISSING' and return the 0th id
        return all_tracks


def remove_punctuation(text_input):
    text_input = str(text_input)  # avoid float errors in applymap()
    text_input = re.sub(r'&', 'and', text_input)  # replaces & with 'and'
    text_input = re.compile(r'[^a-zA-Z 0-9]').sub('', text_input)
    return text_input.lower().strip()


def clean_text(text_input):
    text_input = str(text_input)  # avoid float errors in applymap()
    text_input = text_input.strip().lower()
    text_input = re.sub(r'&', 'and', text_input)  # replaces & with 'and'
    text_input = re.sub(r'and.+', '', text_input)  # removes text after the 'and'
    text_input = re.compile(r'the').sub('', text_input)  # remove all 'the' (maybe just need the 1st word?)
    text_input = re.sub(r',.+', '', text_input)  # removes all misc artists, after comma 
    text_input = re.sub(r'(?:feat).+', '', text_input)  # removes all misc artists, after 'feat' 
    text_input = re.sub(r'\(.+', '', text_input)  # removes text after first bracket
    text_input = re.sub(r'\-.+', '', text_input)  # removes text after first dash
    text_input = re.compile(r'[^a-zA-Z 0-9]').sub('', text_input)  # remove punctuation
    text_input = re.sub(' +', ' ', text_input)  # remove multiple spaces
    return text_input.strip()

In [10]:
%%time
##### SQL 8+M
"""
    SELECT * FROM tracks
    JOIN r_track_artist ON tracks.id = r_track_artist.track_id
    JOIN artists ON r_track_artist.artist_id = artists.id
    JOIN audio_features ON audio_features.id = tracks.audio_feature_id
"""
dtypes = {
    'key': 'Int16', 'mode': 'Int16', 'time_signature': 'Int16', 'tempo': 'float32', 
    'acousticness': 'float32', 'danceability': 'float32', 'duration_ms': 'Int64',  
    'energy': 'float32', 'instrumentalness': 'float32', 'liveness': 'float32', 
    'loudness': 'float32', 'speechiness': 'float32', 'valence': 'float32'
} 
df_SQL = pd.read_csv('all_audio_features_sql.csv', dtype=dtypes)

# import genre and release date data
"""
    SELECT tracks.id AS id, release_date, genre_id as genre FROM tracks
    JOIN r_albums_tracks ON tracks.id = r_albums_tracks.track_id
    JOIN albums ON r_albums_tracks.album_id = albums.id
    JOIN r_track_artist ON tracks.id = r_track_artist.track_id
    JOIN r_artist_genre ON r_track_artist.artist_id = r_artist_genre.artist_id
"""
df_genre = pd.read_csv('SQL_track_release_date_and_genre.csv')
df_genre['genre_count'] = df_genre.groupby('genre')['genre'].transform('count')  # add a count column
df_TEMP = df_genre.copy()  # create temp df, sort by most common genre, merge with SQL data
df_TEMP = df_TEMP.drop_duplicates(['id']).sort_values('genre_count', ascending=False).drop_duplicates(['id']).reset_index(drop=True)
df_SQL = df_SQL.merge(df_TEMP, on='id')

# now format and save df_genre
# NOTE: slightly different then old method, counts multiple instances of a song with multiple artists (for each artist)
df_genre = df_genre[['genre', 'genre_count']].sort_values('genre_count', ascending=False).drop_duplicates(['genre']).reset_index(drop=True)

# formatting
df_SQL['release_date'] = pd.to_datetime(df_SQL['release_date'], unit='ms', origin='unix', errors = 'coerce')
df_SQL = df_SQL.rename({
    'name:1': 'artist',
    'name': 'song',
    'duration:1': 'duration_ms'
}, axis=1)[[
    'id', 'song', 'artist', 'genre', 'release_date',
    'acousticness', 'danceability', 'duration_ms', 
    'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 
    'mode', 'speechiness', 'tempo','time_signature', 'valence'
]].reset_index(drop=True)

# NOTE: DO NOT DROP DUPLICATES YET, NEED FOR LOOKUP WITH B100

# save files as pickle
df_genre.to_pickle('df_genre.pickle')
df_SQL.to_pickle('df_SQL.pickle')

Wall time: 3min 14s


In [3]:
%%time
##### Spotify 1.2M+ Songs
url_1M_songs = r'D:\RYERSON\820\Datasets\Spotify 1.2M+ Songs\tracks_features.csv'
dtypes = {
    'key': 'Int16', 'mode': 'Int16', 'time_signature': 'Int16', 'tempo': 'float32', 
    'acousticness': 'float32', 'danceability': 'float32', 'duration_ms': 'Int64',  
    'energy': 'float32', 'instrumentalness': 'float32', 'liveness': 'float32', 
    'loudness': 'float32', 'speechiness': 'float32', 'valence': 'float32'
} 
df_1M_songs = pd.read_csv(url_1M_songs, dtype=dtypes)

# doesn't have genre (would take 1000+ to get using API, will stay NA)
df_1M_songs['genre'] = pd.NA

# explode artists
df_1M_songs['artists'] = df_1M_songs['artists'].apply(literal_eval) #convert to list type
df_1M_songs = df_1M_songs.explode('artists', ignore_index=True)

# formatting
df_1M_songs['release_date'] = pd.to_datetime(df_1M_songs['release_date'], errors = 'coerce')
df_1M_songs = df_1M_songs.rename({
    'name': 'song',
    'artists': 'artist'
}, axis=1)[[
    'id', 'song', 'artist', 'genre', 'release_date',
    'acousticness', 'danceability', 'duration_ms', 
    'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 
    'mode', 'speechiness', 'tempo','time_signature', 'valence'
]].reset_index(drop=True)

# NOTE: DO NOT DROP DUPLICATES YET, NEED FOR LOOKUP WITH B100

# save files as pickle
df_1M_songs.to_pickle('df_1M_songs.pickle')

Wall time: 20 s


In [36]:
%%time
##### Billboard Top 100 Historical Data
url_B100 = r'D:\RYERSON\820\Datasets\Billboard The Hot 100 Songs\charts.csv'
dtypes_timeseries = {
    'rank': 'Int16', 'last-week': 'Int16', 'peak-rank': 'Int16', 'weeks-on-board': 'Int16'
}
df_B100 = pd.read_csv(url_B100, dtype=dtypes_timeseries)
df_B100['date'] = pd.to_datetime(df_B100['date'])

# Unique Songs from The Billboard 100 Dataset
df_B100_songs = df_B100[['song', 'artist']].drop_duplicates().sort_values(['artist', 'song']).reset_index(drop=True)

# save files as pickle
df_B100.to_pickle('df_B100.pickle')
df_B100_songs.to_pickle('df_B100_songs.pickle')

Wall time: 806 ms


# Merge into Working Datasets

In [149]:
%%time
# reload data
df_genre = pd.read_pickle('df_genre.pickle')
df_SQL = pd.read_pickle('df_SQL.pickle')
df_1M_songs = pd.read_pickle('df_1M_songs.pickle')
df_B100 = pd.read_pickle('df_B100.pickle')
df_B100_songs = pd.read_pickle('df_B100_songs.pickle')

Wall time: 4.7 s


In [100]:
%%time
# merge SQL and CSV AF to get df_10M
ids_SQL = set(df_SQL['id'].to_list())
df_TEMP = df_1M_songs.copy()  
df_TEMP = df_TEMP[~df_TEMP.id.isin(ids_SQL)]  # drop songs already in SQL before merge
df_10M = pd.concat([df_SQL, df_TEMP]).reset_index(drop=True)

# drop duplicate song/artist (exact matches only)
df_10M = df_10M.sort_values('release_date').drop_duplicates(['song', 'artist']).reset_index(drop=True)

Wall time: 29.7 s


In [102]:
%%time
# add approx song and artist columns for approx match
df_10M['approx_song'] = df_10M[['song']].applymap(clean_text)
df_10M['approx_artist'] = df_10M[['artist']].applymap(clean_text)

Wall time: 2min 20s


In [150]:
%%time
# add approx song and artist columns to B100 for merging
df_B100_songs['approx_song'] = df_B100_songs[['song']].applymap(clean_text)
df_B100_songs['approx_artist'] = df_B100_songs[['artist']].applymap(clean_text)

Wall time: 417 ms


In [152]:
%%time
# merge B100 with 10M to get AF, where available
df_B100_songs = df_B100_songs.merge(df_10M, on=['approx_song', 'approx_artist'], how='left')

Wall time: 11 s


In [154]:
%%time
# drop blank artist+song
df_B100_songs = df_B100_songs[~((df_B100_songs.approx_song == '') & (df_B100_songs.approx_artist == ''))]

# sort by exact matches, then release_date
df_B100_songs['EXACT_MATCH'] = (df_B100_songs['song_x'] == df_B100_songs['song_y'])*1 + (df_B100_songs['artist_x'] == df_B100_songs['artist_y'])*1

# drop duplicates (keep exact matches, or oldest song)
df_B100_songs = (
    df_B100_songs.sort_values('release_date')
    .sort_values('EXACT_MATCH', ascending=False)
    .dropna(subset=['song_x', 'artist_x'], axis='rows')
    .drop_duplicates(['song_x', 'artist_x'])
    .reset_index(drop=True)
)

Wall time: 295 ms


In [169]:
# reformat dataframe
df_B100_songs = df_B100_songs.rename({'song_x': 'song', 'artist_x': 'artist'}, axis=1)[[
    'id', 'song', 'artist', 'genre', 'release_date',
    'acousticness', 'danceability', 'duration_ms', 
    'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 
    'mode', 'speechiness', 'tempo','time_signature', 'valence'
]]

In [171]:
# save intermediate file
df_B100_songs.to_pickle('df_B100_songs_PRE_API.pickle')

# Spotify API - GET missing data

In [323]:
# reload data if required
df_B100_songs = pd.read_pickle('df_B100_songs_PRE_API.pickle')

# number missing ids
df_B100_songs[df_B100_songs.id.isnull()].shape[0]

6902

##### get a temporary authorization token from: https://developer.spotify.com/console/get-search-item

In [324]:
# input the temporary token
TEMP_TOKEN = input('Enter token: ')

# create a spotify object
spotify = spotipy.Spotify(auth=TEMP_TOKEN)

Enter token: BQDryQ2lIpWoqbIaZp36XPO_KFxlYmh1uMBwjHf7xlASedyb403Adg_Hiboaesop0vHhDZgq887jfVNUWcODt7YublktB1naCo78fr_Bxu54LJZYRiy_TA4ejEFnAvrFa2uGRfOOxJcwFBzibemYMjjITPhuZ9Po-dSeHjLGhq58


In [328]:
%%time
# loop to get id where missing

counter = 0
id_from_API = set()

for i, row in df_B100_songs.iterrows():
    
    if i < 0:  # where we timed out last time
        continue
        
    if counter % 100 == 0:
        print(counter, end=' ')
    if counter % 1000 == 0:
        print()
        df_B100_songs.to_pickle('df_B100_songs_AF_TEMP.pickle') # save temp file
    
    counter += 1
    
    if not df_B100_songs.iloc[[i]].isnull()['id'].values[0]:
        continue    
    
    # these are the actual song and artist from the Billboard Hot 100
    song = df_B100_songs.loc[df_B100_songs.index[i], 'song']
    artist = df_B100_songs.loc[df_B100_songs.index[i], 'artist']
    
    # get all track info from Spotify API matching 'song' and 'artist'
    all_track_info = find_all_tracks(song, artist)
    
    # restart loop if there are no results
    if all_track_info == 'MISSING':
        continue
    
    # first subloop - check for direct matched
    is_exact_match = False
    id_found = False
    
    for track_info in all_track_info:
        temp_id = track_info[0]
        temp_song = track_info[1]
        temp_artist = track_info[2]
        
        # if there is an exact text match
        is_exact_match = remove_punctuation(temp_song) == remove_punctuation(song) and \
                         remove_punctuation(temp_artist) == remove_punctuation(artist)
        
        # continue subloop
        if is_exact_match:
            df_B100_songs.loc[df_B100_songs.index[i], 'id'] = temp_id
            id_from_API.add(temp_id)
            id_found = True
            break
    
    # if we found the id, go to the next row item, else check for approx matches
    if not id_found:
      
        is_probable_match = False    
            
        # second subloop - check for indirect matches (shouldn't run if direct match occurs)
        for track_info in all_track_info:
            temp_id = track_info[0]
            temp_song = track_info[1]
            temp_artist = track_info[2]

            # if there is a probable text match
            is_probable_match = clean_text(temp_song) == clean_text(song) and \
                                clean_text(temp_artist) == clean_text(artist)

            if is_probable_match:
                df_B100_songs.loc[df_B100_songs.index[i], 'id'] = temp_id
                id_from_API.add(temp_id)
                break
            

df_B100_songs.to_pickle('df_B100_songs_AF_COMPLETE.pickle')

0 
100 200 300 400 500 600 700 800 900 1000 
1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 
2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 
3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 
4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 
5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 
6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 
7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 
8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 
9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 
10100 10200 10300 10400 10500 10600 10700 10800 10900 11000 
11100 11200 11300 11400 11500 11600 11700 11800 11900 12000 
12100 12200 12300 12400 12500 12600 12700 12800 12900 13000 
13100 13200 13300 13400 13500 13600 13700 13800 13900 14000 
14100 14200 14300 14400 14500 14600 14700 14800 14900 15000 
15100 15200 15300 15400 15500 15600 15700 15800 15900 16000 
16100 16200 16300 16400 16500 16600 16700 16800 16900 17000 
17100 17200 17300 17400 17500 17600 17700 17800 17900 18000 
18100 1

In [329]:
# new ids
len(id_from_API)

2762

In [331]:
# number of songs in new ids (should match len(id_from_API))
df_B100_songs[df_B100_songs.id.isin(id_from_API)].id.count()

2762

In [330]:
# still missing ids
df_B100_songs[df_B100_songs.id.isnull()].shape[0]

4140

In [334]:
%%time
# loop to get genre, release_date, and audio features

# ordered list of genres for choosing the best genre
list_of_ordered_genres = list(df_genre.genre)

how_many_passes = 0  # just for QA / error checking
counter = 0

for i, row in df_B100_songs.iterrows():
    # not sure if this is too many GETs...
    
    if i < 0:  # where we timed out last time
        continue
        
    if counter % 100 == 0:
        print(counter, end=' ')
    if counter % 1000 == 0:
        print()
        df_B100_songs.to_pickle('df_B100_songs_AF2_TEMP.pickle') # save temp file
    
    counter += 1
    
    track_id = df_B100_songs['id'].iloc[i]
    
    # if we don't have a new ID, skip the entry (it's not on Spotify)
    if track_id not in id_from_API:
        continue
    
    # Get Audio Features - 1st GET request
    list_of_features = [
        'acousticness', 'danceability', 'duration_ms', 'energy',
        'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
        'speechiness', 'tempo', 'time_signature', 'valence'
    ]
    
    temp_audio_features = spotify.audio_features(track_id)
    for key in list_of_features:
        df_B100_songs.loc[i, key] = temp_audio_features[0][key]

    # Get Release Date - 2nd GET request
    track_info = spotify.track(track_id)
    df_B100_songs.loc[i, 'release_date'] = track_info['album']['release_date']

    # Get Release Date Genre - 3rd GET request
    artist_id = track_info['artists'][0]['id']
    artist_info = spotify.artist(artist_id)
    list_of_artist_genres = artist_info['genres']

    try:
        most_common_genre = list_of_artist_genres[0] # default to first genre
        if len(list_of_artist_genres) == 1:
            pass
        else:
            for genre in list_of_ordered_genres:
                if genre in list_of_artist_genres:
                    most_common_genre = genre
                    break
        df_B100_songs.loc[i, 'genre'] = most_common_genre
    except:
        how_many_passes += 1
        pass  # didn't have any genres (or other error), move on

    
df_B100_songs.to_pickle('df_B100_songs_AF2_COMPLETE.pickle')

0 
100 200 300 400 500 600 700 800 900 1000 
1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 
2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 
3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 
4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 
5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 
6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 
7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 
8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 
9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 
10100 10200 10300 10400 10500 10600 10700 10800 10900 11000 
11100 11200 11300 11400 11500 11600 11700 11800 11900 12000 
12100 12200 12300 12400 12500 12600 12700 12800 12900 13000 
13100 13200 13300 13400 13500 13600 13700 13800 13900 14000 
14100 14200 14300 14400 14500 14600 14700 14800 14900 15000 
15100 15200 15300 15400 15500 15600 15700 15800 15900 16000 
16100 16200 16300 16400 16500 16600 16700 16800 16900 17000 
17100 17200 17300 17400 17500 17600 17700 17800 17900 18000 
18100 1

In [343]:
# no audio features missing
df_B100_songs[df_B100_songs.id.isin(id_from_API)].key.isnull().sum()

0

In [347]:
(
    df_B100_songs.shape[0], 
    df_B100_songs.shape[0] - df_B100_songs.key.isnull().sum(), 
    df_B100_songs.shape[0] - df_B100_songs.genre.isnull().sum()
)
# all, with AF, with genre

(29677, 25537, 23184)

# Merge and Save Datasets

In [4]:
%%time
# reimport data
df_genre = pd.read_pickle('df_genre.pickle')
df_SQL = pd.read_pickle('df_SQL.pickle')
df_1M_songs = pd.read_pickle('df_1M_songs.pickle')
df_B100 = pd.read_pickle('df_B100.pickle')
df_B100_songs = pd.read_pickle('df_B100_songs_AF2_COMPLETE.pickle')  # after API GET requests

Wall time: 9.33 s
